# Riskiness R (list)
### 計算每支ETF的Riskiness R，並儲存

In [1]:
import numpy as np
from numpy import linspace, hstack
import pandas as pd
import time
import datetime as dt
import os 
from scipy.optimize import fsolve
from scipy import stats
from scipy.stats.kde import gaussian_kde
from scipy.stats import norm
from sympy import *
import math
from sklearn.neighbors.kde import KernelDensity

### 過濾資料
### 共32組，其中8組不足36筆，只取24組計算

In [2]:
#cwd = os.getcwd()
data_path = "./ETF_data_set/RE";

c = 0
for subdir in os.listdir(data_path):
    subdir_path = os.path.join(data_path, subdir)
    for file in os.listdir(subdir_path):
        file_path = os.path.join(subdir_path, file)
        df = pd.read_csv(file_path)
        df = df.dropna()
        #print(file)
        print(file_path)
        #print('共',df.shape[0],'筆資料')
        if df.shape[0] == 36:
            c = c + 1

print('\n')
print('有完整36筆資料的有',c,'組')

./ETF_data_set/RE/Alternatives ETF List (35)/Alternatives ETF List (35)_return.csv
./ETF_data_set/RE/Healthcare Equity ETF List (49)/Healthcare Equity ETF List (49)_return.csv
./ETF_data_set/RE/Developed Markets ETF List 1 (100)/Developed Markets ETF List 1 (100)_return.csv
./ETF_data_set/RE/Target Maturity Date Corporate Bond ETF List (24)/Target Maturity Date Corporate Bond ETF List (24)_return.csv
./ETF_data_set/RE/Global ETF List (72)/Global ETF List (72)_return.csv
./ETF_data_set/RE/Municipal Bond ETF List (29)/Municipal Bond ETF List (29)_return.csv
./ETF_data_set/RE/Junk ETF List (43)/Junk ETF List (43)_return.csv
./ETF_data_set/RE/Commodity ETF List (125)/Commodity ETF List (125)_return.csv
./ETF_data_set/RE/Volatility ETF List (18)/Volatility ETF List (18)_return.csv
./ETF_data_set/RE/Currency ETF List (36)/Currency ETF List (36)_return.csv
./ETF_data_set/RE/Developed Markets ETF List 2 (100)/Developed Markets ETF List 2 (100)_return.csv
./ETF_data_set/RE/Industrials Equity ET

# functions

In [3]:
def get_data_from_csv(file):
    df = pd.DataFrame()
    if os.path.isfile(file):
        df = pd.read_csv(file)
        df = df.dropna()
        df.drop(df.columns[[0]], axis=1, inplace=True)
    return df

def cal_sharp_ratio(data):
    total = len(data)
    data = np.log(data)
    mean = (data.mean())
    std = data.std()
    skew = data.skew()
    kurt = data.kurt()
    var = -1.96
    Z = var + (skew * ((var**2) - 1) / 6) + (kurt * ((var**3) - 3*var) / 24) - ((skew**2) * (2*(var**3) - 5*var) / 36)
    adj_var = (-0.5)*(std**2)*52-Z*std*math.sqrt(total)
    cumprod_return = np.exp(data).cumprod()[-1]
    sharpe_ratio = cumprod_return / adj_var
    return sharpe_ratio

def cal_omega(data,L):
    probDensityFun = gaussian_kde(data)
    x = np.linspace(-0.5,0.5,10000)        #將數據用平滑的方式算單點機率
    p = (probDensityFun(x) / probDensityFun(x).sum())
    p = p.cumsum()
    p = pd.DataFrame(p,index=x)
    omega = ((1-p).loc[L/100/52:].sum()) / (p.loc[:L/100/52].sum())
    return omega[0]

def f1(risk,returns):
    return sum( np.exp(-1*returns/risk) ) - len(returns)

def cal_riskiness_r(data):
    rfrate = 2.45785 / 12 / 100  #月報酬
    data = data - rfrate
    guess = 10**(-5)
    while (guess<1000):
        risk = fsolve(f1,guess,data)
        if risk[0] != guess:
            break
        guess = guess*10
    return np.exp(-1/risk[0])

def cal_index_and_save_file(group_name, etf_data):
    etf_symbols = list(etf_data.columns)

    list_riskinessr = []
    list_omega = []

    for i in range(etf_data.shape[1]):
        etf_title = etf_symbols[i]
        etf_return = etf_data.iloc[:,i]

        r = cal_riskiness_r(etf_return)
        list_riskinessr.append(r)

        #L = USG3M.loc[etf_return.index[-1]]
        #L = 2.45785 / 12 / 100  #月報酬
        #omega = cal_omega(etf_return,L)
        #list_omega.append(omega)

    saved_file = './RiskinessR/' + group_name + '.csv'
    #df2 = pd.DataFrame({'ETF':etf_symbols, 'RiskinessR':list_riskinessr, 'Omega':list_omega})
    df2 = pd.DataFrame({'ETF':etf_symbols, 'RiskinessR':list_riskinessr})
    df2.to_csv(saved_file, sep=',')


# 一組一組算風險指標 R

In [4]:
data_path = "./ETF_data_set/RE";
for subdir in os.listdir(data_path):
    subdir_path = os.path.join(data_path,subdir)
    for file in os.listdir(subdir_path):
        file_path = os.path.join(subdir_path, file)
        etf_data = get_data_from_csv(file_path)
        print(etf_data.shape)
        group_name = subdir
        print(group_name)
        cal_index_and_save_file(group_name, etf_data)

(36, 21)
Alternatives ETF List (35)


/Users/alexis/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/Users/alexis/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:163: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)


(36, 34)
Healthcare Equity ETF List (49)
(36, 80)
Developed Markets ETF List 1 (100)
(36, 16)
Target Maturity Date Corporate Bond ETF List (24)
(4, 40)
Global ETF List (72)
(36, 14)
Municipal Bond ETF List (29)
(36, 20)
Junk ETF List (43)
(36, 78)
Commodity ETF List (125)
(36, 8)
Volatility ETF List (18)
(36, 25)
Currency ETF List (36)
(36, 58)
Developed Markets ETF List 2 (100)
(7, 26)
Industrials Equity ETF List (37)
(36, 42)
Financials Equity ETF List (51)
(36, 27)
Broad Asia ETF List (44)
(36, 7)
Preferred Stock ETF List (12)
(4, 78)
Emerging Asia Pacific ETF List (114)
(36, 60)
Energy Equity ETF List (80)
(36, 44)
Emerging Markets ETF List (79)
(36, 8)
Telecom Equity ETF List (10)
(36, 47)
Technology Equity ETF List (75)
(36, 16)
Consumer Staples Equity ETF List (26)
(7, 61)
Developed Asia Pacific ETF List (108)
(36, 52)
Materials Equity ETF List (62)
(36, 35)
Real Estate ETF List (48)
(36, 17)
Utilities Equity ETF List (24)
(36, 22)
Consumer Discretionary Equity ETF List (31)
(28